# Continuous Control

---

Congratulations for completing the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control an agent in a more challenging environment, where the goal is to train a creature with four arms to walk forward.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Crawler.app"`
- **Windows** (x86): `"path/to/Crawler_Windows_x86/Crawler.exe"`
- **Windows** (x86_64): `"path/to/Crawler_Windows_x86_64/Crawler.exe"`
- **Linux** (x86): `"path/to/Crawler_Linux/Crawler.x86"`
- **Linux** (x86_64): `"path/to/Crawler_Linux/Crawler.x86_64"`
- **Linux** (x86, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86"`
- **Linux** (x86_64, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86_64"`

For instance, if you are using a Mac, then you downloaded `Crawler.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Crawler.app")
```

In [ ]:
env = UnityEnvironment(file_name='Crawler_Linux/Crawler.x86_64')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Actor(nn.Module):

    def __init__(self, inputs, outputs, hidden_layers=(128, 256), device='cpu'):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(inputs, hidden_layers[0])
        f1 = 1 / np.sqrt(self.fc1.weight.data.size()[0])
        nn.init.uniform_(self.fc1.weight.data, -f1, f1)
        nn.init.uniform_(self.fc1.bias.data, -f1, f1)

        self.bn1 = nn.LayerNorm(hidden_layers[0])
        self.hidden_layers = nn.ModuleList()
        self.hidden_norms = nn.ModuleList()
        for i in range(len(hidden_layers)-1):
            self.hidden_layers.append(
                nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            fn = 1 / np.sqrt(self.hidden_layers[i].weight.data.size()[0])
            nn.init.uniform_(self.hidden_layers[i].weight.data, -fn, fn)
            nn.init.uniform_(self.hidden_layers[i].bias.data, -fn, fn)
            self.hidden_norms.append(nn.LayerNorm(hidden_layers[i+1]))

        self.output = nn.Linear(hidden_layers[-1], outputs)
        self.to(torch.device(device))

    def forward(self, X):
        X = F.relu(self.bn1(self.fc1(X)))
        for i in range(len(self.hidden_layers)):
            X = F.relu(self.hidden_norms[i](self.hidden_layers[i](X)))

        return torch.tanh(self.output(X))


class Critic(nn.Module):

    def __init__(self, inputs, outputs, hidden_layers=(128, 256), device='cpu'):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(inputs, hidden_layers[0])
        f1 = 1 / np.sqrt(self.fc1.weight.data.size()[0])
        nn.init.uniform_(self.fc1.weight.data, -f1, f1)
        nn.init.uniform_(self.fc1.bias.data, -f1, f1)

        self.bn1 = nn.LayerNorm(hidden_layers[0])
        self.hidden_layers = nn.ModuleList()
        self.hidden_norms = nn.ModuleList()
        for i in range(len(hidden_layers)-1):
            self.hidden_layers.append(
                nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            fn = 1 / np.sqrt(self.hidden_layers[i].weight.data.size()[0])
            nn.init.uniform_(self.hidden_layers[i].weight.data, -fn, fn)
            nn.init.uniform_(self.hidden_layers[i].bias.data, -fn, fn)
            self.hidden_norms.append(nn.LayerNorm(hidden_layers[i+1]))
        self.output = nn.Linear(hidden_layers[-1], outputs)
        self.to(torch.device(device))

    def forward(self, X1, X2):
        X = torch.cat([X1, X2], 1)  # concatenate states and actions by columns
        X = F.relu(self.bn1(self.fc1(X)))
        for i in range(len(self.hidden_layers)):
            X = F.relu(self.hidden_norms[i](self.hidden_layers[i](X)))

        return self.output(X)


In [ ]:
from collections import deque
import random


class ReplayBuffer:

    def __init__(self, cap=512):
        self.buffer = deque(maxlen=cap)

    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array(reward), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        states, actions, rewards, next_states, dones = zip(
            *random.sample(self.buffer, batch_size))
        return np.vstack(states), np.vstack(actions), np.vstack(rewards), np.vstack(next_states), np.vstack(dones)

    def __len__(self):
        return len(self.buffer)


In [ ]:
def hard_copy(net: nn.Module, target_net: nn.Module):
    for param, target_param in zip(net.parameters(), target_net.parameters()):
        target_param.data.copy_(param.data)


def soft_copy(net: nn.Module, target_net: nn.Module, tau=1e-2):
    for param, target_param in zip(net.parameters(), target_net.parameters()):
        target_param.data.copy_(
            param.data * tau + target_param.data * (1.0 - tau))


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline


class OUNoise(object):
    def __init__(self, action_space, mu=0.0, theta=0.15, max_sigma=0.3, min_sigma=0.1, decay_period=100000):
        self.mu = mu
        self.theta = theta
        self.sigma = max_sigma
        self.max_sigma = max_sigma
        self.min_sigma = min_sigma
        self.decay_period = decay_period
        self.action_dim = action_space
        self.low = -1.0
        self.high = 1.0
        self.reset()

    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu

    def evolve_state(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * \
            np.random.randn(self.action_dim)
        self.state = x + dx
        return self.state

    def get_action(self, action, t=0):
        ou_state = self.evolve_state()
        self.sigma = self.max_sigma - \
            (self.max_sigma - self.min_sigma) * min(1.0, t / self.decay_period)
        return np.clip(action + ou_state, self.low, self.high)


In [ ]:
def plot_rewards(rewards, average=None, goal=30, title=None, filename=None):
    plt.plot(rewards, label='Reward')
    if average is not None:
        plt.plot(average, label='Average Reward')
    plt.plot([goal] * len(rewards), linestyle='--', label='Goal')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.title(title)
    plt.legend()

    if filename is not None:
        plt.savefig(filename)

    plt.show()

In [ ]:
import torch.optim as optim
import gc

NUM_EPISODES = 10_000
BATCH_SIZE = 128
NET_UPD = 10
GAMMA = 0.99
TAU = 1e-2
ACTOR_LR = 1e-4
CRITIC_LR = 1e-3
BUFFER_LEN = 1_000_000

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device in use: {device}')

num_states = env_info.vector_observations.shape[1]
num_actions = brain.vector_action_space_size

actor = Actor(num_states, num_actions, hidden_layers=(256, 256, 512), device=device)
actor_target = Actor(num_states, num_actions,
                     hidden_layers=(256, 256, 512), device=device)
hard_copy(actor, actor_target)

print(f'Actor {actor}')

critic = Critic(num_states + num_actions, action_size,
                hidden_layers=(256, 256, 512), device=device)
critic_target = Critic(num_states + num_actions, action_size,
                       hidden_layers=(256, 256, 512), device=device)
hard_copy(critic, critic_target)

print(f'Critic {critic}')

criterion = nn.MSELoss()
actor_optim = optim.Adam(actor.parameters(), lr=ACTOR_LR)
critic_optim = optim.Adam(critic.parameters(), lr=CRITIC_LR)

replay_buffer = ReplayBuffer(cap=BUFFER_LEN)

noise = OUNoise(num_actions)


def update_agent():
    states, actions, rewards, next_states, _ = replay_buffer.sample(BATCH_SIZE)
    states = torch.from_numpy(states).float().to(device)
    actions = torch.from_numpy(actions).float().to(device)
    rewards = torch.from_numpy(rewards).float().to(device)
    next_states = torch.from_numpy(next_states).float().to(device)

    Qval = critic.forward(states, actions)
    next_actions = actor_target.forward(next_states)
    nextQ = critic_target.forward(next_states, next_actions.detach())
    Qprime = rewards + GAMMA * nextQ

    critic_loss = criterion(Qval, Qprime)
    actor_loss = -critic.forward(states, actor.forward(states)).mean()

    actor_optim.zero_grad()
    actor_loss.backward()
    actor_optim.step()

    critic_optim.zero_grad()
    critic_loss.backward()
    nn.utils.clip_grad_norm_(critic.parameters(), 1)
    critic_optim.step()

    soft_copy(actor, actor_target)
    soft_copy(critic, critic_target)


@torch.no_grad()
def get_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    action = actor.forward(state)
    return action.cpu().detach().numpy()[0]

# prefill_buffer(partition=100)
print('\n')
episode_rewards = []
avg_rewards = []
total_steps = 0
for e in range(NUM_EPISODES):
    env_info = env.reset(train_mode=True)[brain_name]
    states_all = env_info.vector_observations
    noise.reset()
    step = 0
    ep_reward = 0
    while True:
        step += 1

        actions = np.zeros((num_agents, action_size))
        for i in range(num_agents):
            actions[i] = get_action(states[i])
            actions[i] = noise.get_action(actions[i], step)

        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done

        for i in range(num_agents):
            replay_buffer.push(states[i], actions[i], rewards[i], next_states[i], dones[i])

        states = next_states
        ep_reward += np.mean(rewards)
        if np.any(dones):
            if len(replay_buffer) > BATCH_SIZE:
                for i in range(NET_UPD * num_agents // 2):
                    update_agent()
            break

    episode_rewards.append(ep_reward)
    avg_rewards.append(np.mean(episode_rewards[-10:]))
    total_steps += step
    print(
        f'\rEpisode {e} reward {ep_reward:.3f} and average reward {avg_rewards[-1]:.3f} after {total_steps} steps', end='')
    gc.collect()

    if len(episode_rewards[-30:]) >= 30 and np.min(episode_rewards[-30:]) > 30.:
        print(f'Early stop at episode {e}; Goal has been reached; Mean reward in the last 30 episodes {np.mean(episode_rewards[-30:])}')
        break

In [ ]:
plot_rewards(episode_rewards, average=avg_rewards, title="Training")

In [ ]:
eval_rewards = []
for e in range(10):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations

    ep_reward = 0
    while True:
        state = torch.from_numpy(states).float().unsqueeze(0).to(device)
        actions = actor_target.forward(state)
        actions = actions.cpu().detach().numpy()[0]

        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done

        ep_reward += np.mean(rewards)
        states = next_states
        if np.any(dones):
            break
    
    eval_rewards.append(ep_reward)
    print(f'\rEvaluation #{e+1} finished with reward {ep_reward}', end='')

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
env.brain_names